# Tarea 1 Comprobaciones previas (0,5 puntos)
 Muestra el número de documentos que se almacenan en la colección pedidos

In [2]:
from pymongo import MongoClient

# Establecer conexión con el servidor de MongoDB
cliente = MongoClient('127.0.0.3', 27017)

# Borrar la base de datos 'supermercado'
cliente.drop_database('supermercado')

print("La base de datos 'supermercado' ha sido eliminada correctamente.")


ServerSelectionTimeoutError: 127.0.0.3:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 66110377f920797f923f3daf, topology_type: Unknown, servers: [<ServerDescription ('127.0.0.3', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('127.0.0.3:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

# Tarea 2 Inserciones de registros (0,5 puntos)

### Inserta tres documentos en la colección utilizando un único comando:

- El primero tendrá como ID_Pedido el 79159 e Importe total 3.45
- El segundo tendrá el mismo ID_Pedido (79159), Importe total de 0.23 y Número de items 5.
- El tercero tendrá el mismo ID_Pedido (79159) y Número de items 8.

(Verifica que se han insertado los tres documentos)

In [17]:
# Insertar el primer documento
documento_1 = {
    "ID_Pedido": 79159,
    "Importe_total": 3.45
}

resultado = db["pedidos"].insert_one(documento_1)
print("ID del documento insertado:", resultado.inserted_id)


ID del documento insertado: 65e78114cd1416084cd958cf


In [18]:
# Insertar el segundo documento
documento_2 = {
    "ID_Pedido": 79159,
    "Importe_total": 0.23,
    "Numero_de_items": 5
}

resultado = db["pedidos"].insert_one(documento_2)
print("ID del segundo documento insertado:", resultado.inserted_id)


ID del segundo documento insertado: 65e78117cd1416084cd958d0


In [19]:
# Insertar el tercer documento
documento_3 = {
    "ID_Pedido": 79159,
    "Numero_de_items": 8
}

resultado = db["pedidos"].insert_one(documento_3)
print("ID del tercer documento insertado:", resultado.inserted_id)


ID del tercer documento insertado: 65e7811acd1416084cd958d1


# Tarea 3 Consultas (3,5 puntos)

### 1. Dirección asociada al pedido 34789
Recupera la dirección asociada al pedido 34789. Solo nos interesa recuperar la dirección con todos sus campos

In [20]:
pedido_34789 = db["pedidos"].find_one({"ID_Pedido": 34789})

if pedido_34789:
    direccion_cliente_34789 = pedido_34789.get("Cliente", {}).get("Dirección", {})
    
    print("Datos de dirección del cliente del pedido 34789:")
    print(direccion_cliente_34789)
else:
    print("No se encontró el pedido con ID 34789")


Datos de dirección del cliente del pedido 34789:
{'ID_Dirección': 29, 'Calle': 'Calle de Santa Inés', 'Número': 5, 'Código Postal': 28231, 'Localidad': 'Las rozas', 'Provincia': 'Madrid'}


### 2. Número de pedidos que NO disponen de dirección del cliente
Calcula el número de pedidos que no disponen de dirección del cliente.

In [21]:
num_pedidos_sin_direccion = db["pedidos"].count_documents({"Direccion": {"$exists": False}})
print("Número de pedidos sin dirección del cliente:", num_pedidos_sin_direccion)


Número de pedidos sin dirección del cliente: 3014


### 3. Datos de cliente con el pedido de menor Importe total
Recupera los datos del cliente con el pedido de menor Importe total, incluyendo su ID_Pedido, Importe total y todos los datos del cliente con su dirección asociada.

In [16]:
import pymongo

pedido_menor_importe = db["pedidos"].find_one({}, sort=[("Importe total", 1)])

if pedido_menor_importe:
    numero_pedido_menor_importe = pedido_menor_importe["ID_Pedido"]
    print("Número de pedido con el menor importe total:", numero_pedido_menor_importe)

    pedido_completo_menor_importe = db["pedidos"].find_one({"ID_Pedido": numero_pedido_menor_importe})

    if pedido_completo_menor_importe:
        print("\nDatos completos del pedido con el menor importe total:")
        for key, value in pedido_completo_menor_importe.items():
            print(f"{key}: {value}")
    else:
        print("No se encontró ningún pedido con el número", numero_pedido_menor_importe)
else:
    print("No se encontraron pedidos en la base de datos.")


Número de pedido con el menor importe total: 79159

Datos completos del pedido con el menor importe total:
_id: 65e77431dae9ce30ef5b4e2a
ID_Pedido: 79159
Importe_total: 3.45


### 4. Pedidos cuyo Importe total está entre 2 y 2.1 euros
Recupera los ID_Pedido de aquellos pedidos cuyo Importe total está entre 2 y 2.1 euros.

In [14]:
pedidos_rango_importe = db["pedidos"].find({"Importe total": {"$gte": 2, "$lte": 2.1}}, {"ID_Pedido": 1})
for pedido in pedidos_rango_importe:
    print("ID_Pedido:", pedido["ID_Pedido"])


ID_Pedido: 67693
ID_Pedido: 67960
ID_Pedido: 11047


### 5. Pedidos que incluyen Pan y Jabón entre sus ítems
Recupera los ID_Pedido de aquellos pedidos que incluyen Pan y Jabón entre sus ítems.

In [15]:
pedidos_pan_jabon = db["pedidos"].find({"Items": {"$all": ["Pan", "Jabón"]}}, {"ID_Pedido": 1})
for pedido in pedidos_pan_jabon:
    print("ID_Pedido:", pedido["ID_Pedido"])